# Food Image Classification for the course _Foundations of Deep Learning_.

Professors:
Paolo Napoletano
Marco Buzzelli

Tutor:
Mirko Agarla

## Notebook for Data preprocessing and Data Augmentation

In [1]:
!pip install split-folders # install the package to split the images in TRAIN and TEST folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# IMPORT MODULES
import os
import fnmatch # to count number of image per folder easily
import random # to select random images from a folder
import numpy as np
import pandas as pd
import splitfolders # needed to split the images in TRAIN and TEST folders
from shutil import copyfile # to import "copyfile"
from matplotlib import pyplot as plt

# CV2
import cv2
from google.colab.patches import cv2_imshow 

In [4]:
# Link Google Drive account
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
# IMPORT UTILS.PY

# 1.Insert the directory
import sys
sys.path.insert(0,"/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food")

# 2.Import your module or file
import utils

In [11]:
# CONSTANT
# SCALE_PERCENT = 6 # resizing percentage
SEED_VALUE = 42
num_classes = 9 # number of output classes

# PATH TO THE DATA
ORIGINAL_DATA = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset"
PREPROCESSED_DATA = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDatasetPreprocessed"
DIR_TRAIN_TEST_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test' # path contenente i sottoinsiemi di train, test

# 1.Loading the dataset and Data Exploration
In this section we will describe the data at hand.

> ## Exploratory Data Analysis (EDA)

In [7]:
# HOW MANY FOLDERS (CLASSES) DO WE HAVE? WHICH CLASSES DO WE HAVE?
lst = os.listdir(ORIGINAL_DATA) # your directory path
file_names = lst
number_files = len(file_names[1:])
print("Class names:", file_names[1:])
print("Number of classes: ", number_files)

Class names: ['french_fries', 'caprese_salad', 'pizza', 'greek_salad', 'sashimi', 'hot_dog', 'caesar_salad', 'hamburger', 'sushi']
Number of classes:  9


In [8]:
# COUNT THE NUMBER OF ELEMENTS IN EACH CLASS FOLDERS

# FOLDER PATH
count = 0
hist_dict = {}

# ITERATE DIRECTORY
for path in os.listdir(ORIGINAL_DATA):
    # CHECK IF CURRENT PATH IS A FILE
    if os.path.isfile(os.path.join(ORIGINAL_DATA, path)):
      count += 0
    else:
      # IF IS NOT A FILE, IS A DIRECTORY. GET THE FOLDER NAME
      number_of_elements_per_folder = len([entry for entry in os.listdir(ORIGINAL_DATA + "/" + path)])
      print(f"The folder called {path} has a number of element(s) equal to:", number_of_elements_per_folder)
      count += 1
      hist_dict[path] = number_of_elements_per_folder

print()
print('Number of folders:', count, "different classes of food.")

The folder called french_fries has a number of element(s) equal to: 181
The folder called caprese_salad has a number of element(s) equal to: 15
The folder called pizza has a number of element(s) equal to: 299
The folder called greek_salad has a number of element(s) equal to: 24
The folder called sashimi has a number of element(s) equal to: 40
The folder called hot_dog has a number of element(s) equal to: 31
The folder called caesar_salad has a number of element(s) equal to: 27
The folder called hamburger has a number of element(s) equal to: 238
The folder called sushi has a number of element(s) equal to: 124

Number of folders: 9 different classes of food.


In [12]:
# SHOW THE DATASET DISTRIBUTION WITH A HISTOGRAM PLOT
plt.figure(figsize=(15, 10))

# CONVERT THE DICT TO DATAFRAME
new = pd.DataFrame([hist_dict]).transpose().reset_index()
new = new.rename(columns={"index": "food_name", 0: "value_count"})
# SORT THE DATAFRAME BASED ON VALUE_COUNT
new = new.sort_values("value_count", ascending=False)
new

# CREATING DATA ON WHICH BAR CHART WILL BE PLOT
x_food_name = list(new['food_name'])
y_value_count = list(new['value_count'])

# CALLING THE FUNCTION TO ADD VALUE LABELS
utils.addlabels(x_food_name, y_value_count)
plt.xticks(rotation=45)

AttributeError: ignored

<Figure size 1080x720 with 0 Axes>

It is clear that this dataset shows an _unbalanced class problem_ that needs to be addressed through _data augmentation_.

## Show some images as example, their original sizes and the format

As it is visible from the image shape, they are of different sizes but all of them contain the full RGB color spectrum. In order to correctly treat them and feed them inside the CNN it is important that a preprocessing step of resizing them will be applied.



> ### DISPLAY SEVERAL IMAGES AT ONCE
Pick some of the images randomly from the dataset, and show them.


In [ ]:
# DECIDE THE NUMBER OF IMAGES PER ROW AND COLUMN TO SHOW
rows = 3
cols = 3

# DEFINE SUBPLOTS BASED ON ROWS AND COLUMNS AND FIGSIZE
fig, ax = plt.subplots(rows, cols, figsize=(9, 7)) # fig = relative to the figure; ax = relative to the axes.
fig.suptitle('Random Image from Each Food Class', fontsize=20) # ADD A TITLE

# CREATE AN EMPTY LIST IN ORDER TO ADD ONLY THE DIRECTORY AND NOT THE FILES
food_dirs = []

# FOR EACH FILE IN ORIGINAL_DATA, CHECK IF IT IS A DIRECTORY AND, IF YES, ADD IT TO THE FOOD_DIRS FOLDER
for filename in os.listdir(ORIGINAL_DATA):
    if os.path.isdir(os.path.join(ORIGINAL_DATA,filename)):
        food_dirs.append(filename)

# PRINT THE FINAL LIST
# print(food_dirs)

# SHOW SOME RANDOM IMAGES FROM THE FOLDERS
for i in range(rows):
    for j in range(cols):
      food_dir = food_dirs[i*cols + j]
      all_files = os.listdir(os.path.join(ORIGINAL_DATA, food_dir))
      rand_img = np.random.choice(all_files)
      img = plt.imread(os.path.join(ORIGINAL_DATA, food_dir, rand_img))
      ax[i][j].imshow(img)
      
      # CREATE COLORS FOR THE BOX AROUND THE TEXT
      ec = (0, .6, .1)
      fc = (0, .7, .2)
      # CREATE THE TEXT ON TOP OF THE IMAGE
      ax[i][j].text(0, -20, food_dir, size=10, rotation=0, ha="left", va="top", 
              bbox=dict(boxstyle="round", ec=ec, fc=fc))
      print(f"The image corresponding to {food_dir} has the shape of:", img.shape)
        
# plt.setp(ax, xticks=[], yticks=[]) # do not show the ticks
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

As it is visible from the image shape, they are of different sizes but all of them contain the full RGB color spectrum. In order to correctly treat them and feed them inside the CNN it is important that a preprocessing step of resizing them will be applied.

# 3.DATA PREPROCESSING

## Resize all the images

#When using the ImageDataGenerator method in keras, the use target_size is used for resizing the images before feed them to the model. So, it is not necessary to resize them in advance!

In [ ]:
# CREATE AN EMPTY LIST TO FILL WITH THE CATEGORIES
CATEGORIES = []

# LOOP THROUGH THE ROOT DIRECTORY, SELECT ONLY THE DIRECTORIES AND SAVE THEIR NAME
for filename in os.listdir(ORIGINAL_DATA):
    if os.path.isdir(os.path.join(ORIGINAL_DATA,filename)):
        CATEGORIES.append(filename)

# SHOW THE CATEGORY NAMES
CATEGORIES

In [ ]:
from os import mkdir
from collections import defaultdict
# CREATE A DIRECTORY FOR THE PROCESSED IMAGES
try:
    os.mkdir(PREPROCESSED_DATA)
except:
    print("Folder already found.")

In [ ]:
# RESIZE THE IMAGES AND SAVE THEM

# CREATE LISTS TO STORE THE IMAGE SIZE
image_size = {}
img_width_list = list()
img_height_list = list()

for i in CATEGORIES:
    # print(f'CATEGORY: {i}')
    # CREATE A DIRECTORY FOR THE PROCESSED IMAGES
    
    try:
        os.mkdir(os.path.join(PREPROCESSED_DATA,i))
    except:
        print("Folder already found.")
        # pass

    path = os.path.join(ORIGINAL_DATA,i)
    
    for img in os.listdir(path):
      if img.endswith(('.png', '.jpg', '.jpeg')):
        img_raw = cv2.imread(os.path.join(ORIGINAL_DATA, i, img))
        img_width, img_height, img_channel = img_raw.shape # get the size of the image

        ##################################################################

        # STORE THE WIDTH
        img_width_list.append(img_width)
        img_width_set = set(img_width_list)

        # STORE THE HEIGHT
        img_height_list.append(img_height)
        img_height_set = set(img_height_list)

        ##################################################################

        # CHECK IF IMG_WIDTH IS IN THE DICTIONARY. IF NOT ADD IT; IF YES +1
        if img_width in image_size:
            image_size[img_width] += 1
        else:
            image_size[img_width] = 1

        # CHECK IF IMG_WIDTH IS IN THE DICTIONARY. IF NOT ADD IT; IF YES +1
        if img_height in image_size:
            image_size[img_height] += 1
        else:
            image_size[img_height] = 1
      else:
        continue

      # RESIZING
      # width1 = int(img_raw.shape[1] * SCALE_PERCENT / 100)
      # height1 = int(img_raw.shape[0] * SCALE_PERCENT / 100)
      width1 = 224
      height1 = 224
      dim1 = (width1, height1)
      img_resized = cv2.resize(img_raw, dim1, interpolation = cv2.INTER_AREA)

      # SAVING THE IMAGES
      cv2.imwrite(os.path.join(PREPROCESSED_DATA, i, f"{img}_RESIZED.jpg"), img_resized)

      # Save Image
      print('Saving the image...')

In [ ]:
# GET THE MINIMUM OF WIDTH
print("Minimum of image width: ", min(set(img_width_list)))

# GET THE MAXIMUM OF WIDTH
print("Maximum of image width: ", max(set(img_width_list)))
print()
# GET THE MINIMUM OF HEIGHT
print("Minimum of image height: ", min(set(img_height_list)))

# GET THE MAXIMUM OF HEIGHT
print("Maximum of image height: ", max(set(img_height_list)))

In [ ]:
# PLOT AN HISTOGRAM OF IMAGE SIZES
plt.figure(figsize=(10,8))
plt.bar(list(image_size.keys()), image_size.values(), color='b')
plt.xlim((0,1000))
plt.ylim((0,40))
plt.show()

## Data Splitting

In [ ]:
# CREATE A DIRECTORY FOR THE SPLITTING
try:
    os.mkdir(DIR_TRAIN_TEST_DATA)
except:
    print("Folder already found.")

In [ ]:
# SPLIT THE IMAGE IN TRAIN AND TEST SETS. 65% TRAIN DATA AND 35% TEST DATA.
splitfolders.ratio(PREPROCESSED_DATA,
                   output= DIR_TRAIN_TEST_DATA,
                   seed = SEED_VALUE,
                   ratio = (.65, .20, .15)) # training, validation, testing

# 4.Data Augmentation on the train set

In this section we will apply different techniques in order to create additional images to be fed, later, into the model.
Why _Data Augmentation_? (INSERT DESCRIPTION)

In [ ]:
# CHECK AGAIN THE NUMBER OF IMAGES FOR EACH CLASSES
# COUNT THE NUMBER OF ELEMENTS IN EACH CLASS FOLDERS

# COUNT IMAGES AND CREATE A DICT
count_processed = 0
hist_dict_processed = {}

# ITERATE DIRECTORY
for path in os.listdir(DIR_TRAIN_TEST_DATA):
  if path == "train":

    print(path)
    for categories in os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path)):
      print("--", categories)

      for images in os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path, categories)):

        # CHECK IF CURRENT PATH IS A FILE
        if os.path.isfile(os.path.join(DIR_TRAIN_TEST_DATA, path, categories)):
          count_processed += 0
        else:
          # IF IS NOT A FILE, IS A DIRECTORY. GET THE FOLDER NAME
          number_of_elements_per_folder = len([entry for entry in os.listdir(DIR_TRAIN_TEST_DATA + "/" + path + "/" + categories)])
          # print(f"The folder called {path} has a number of element(s) equal to:", number_of_elements_per_folder)
          count_processed += 1
          hist_dict_processed[categories] = number_of_elements_per_folder
  else:
    break

print()
# print('Number of folders:', count_processed, "different classes of food.")

In [ ]:
# SHOW THE DATASET DISTRIBUTION WITH A HISTOGRAM PLOT ONLY FOR THE TRAINING SET
plt.figure(figsize=(15, 10))

# CONVERT THE DICT TO DATAFRAME
new_processed = pd.DataFrame([hist_dict_processed]).transpose().reset_index()
new_processed = new_processed.rename(columns={"index": "food_name", 0: "value_count"})

# SORT THE DATAFRAME BASED ON VALUE_COUNT
new_processed = new_processed.sort_values("value_count", ascending=False)
new_processed

# PLOT THE BAR AND THE TICKS
plt.bar(new_processed['food_name'], new_processed['value_count'], color='g')
plt.xticks(rotation=45)

# CREATING DATA ON WHICH BAR CHART WILL BE PLOT
x_food_name = list(new_processed['food_name'])
y_value_count = list(new_processed['value_count'])

# CALLING THE FUNCTION TO ADD VALUE LABELS
utils.addlabels(x_food_name, y_value_count)

It is still visible that the training set has imbalance classes, a problem that has to be faced before creating the Convolutional Neural Network model.

In [ ]:
# DATA AUGMENTATION

# COUNT IMAGES AND CREATE A DICT
count_images = 0

# ITERATE DIRECTORY
for path in os.listdir(DIR_TRAIN_TEST_DATA):
  if path == "train":

    print(path)
    for categories in os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path)):
      print("--", categories)
      # USE https://docs.python.org/3/library/fnmatch.html TO CHECK THE SIZE OF EACH FOLDER
      category_size = len(fnmatch.filter(os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path, categories)), '*.jpg'))

      while category_size < 200:
        

        # PATH TO THE IMAGES
        category_path_folder = fnmatch.filter(os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path, categories)), '*.jpg')
        # print(DIR_TRAIN_TEST_DATA)

        # LOAD A RANDOM IMAGE AND READ IT
        random_image = random.choice(category_path_folder)
        image = cv2.imread(os.path.join(DIR_TRAIN_TEST_DATA, path, categories, random_image))

        # PERFORM DATA AUGMENTATION
        # -------------------------
        # HORIZONTAL FLIP
        img_horiz_flipped = utils.horizontal_flip(image, True)
        # VERTICAL FLIP
        img_vertical_flipped = utils.vertical_flip(image, True)
        # BRIGHTNESS
        img_brightness = utils.brightness(image, 0.5, 1.3)
        # CHANNEL SHIFT
        img_hue_image = utils.hue_image(image, 5)
        
        # SHOW THE IMAGE
        # cv2_imshow(img_horiz_flipped)

        # SAVING THE IMAGES
        cv2.imwrite(os.path.join(DIR_TRAIN_TEST_DATA, path, categories, f"{random_image}_horiz_flipped.jpg"), img_horiz_flipped)
        cv2.imwrite(os.path.join(DIR_TRAIN_TEST_DATA, path, categories, f"{random_image}_vertical_flipped.jpg"), img_vertical_flipped)
        cv2.imwrite(os.path.join(DIR_TRAIN_TEST_DATA, path, categories, f"{random_image}_vertical_flipped.jpg"), img_brightness)
        cv2.imwrite(os.path.join(DIR_TRAIN_TEST_DATA, path, categories, f"{random_image}_vertical_flipped.jpg"), img_hue_image)
        # print(os.path.join(DIR_TRAIN_TEST_DATA, path, categories, f"{random_image}_horiz_flipped.jpg"), img_horiz_flipped)

        # WHAT IS THE CURRENT SIZE OF THE FOLDER?
        category_size = len(fnmatch.filter(os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path, categories)), '*.jpg'))
        print("Category size is (continue until reached 1000 images in each folder):", category_size)

      else:
        # EXIT FROM THIS CATEGORY DIRECTORY IF THE NUMBER OF IMAGES INSIDE IS MORE THAN 200
        continue

  else:
    break

In [ ]:
# CHECK SIZE OF FOLDERS
print("Size of the folder caesar_salad: ",  len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/caesar_salad"), '*.jpg')))
print("Size of the folder caprese_salad: ", len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/caprese_salad"), '*.jpg')))
print("Size of the folder french_fries: ",  len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/french_fries"), '*.jpg')))
print("Size of the folder greek_salad: ",   len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/greek_salad"), '*.jpg')))
print("Size of the folder hamburger: ",     len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/hamburger"), '*.jpg')))
print("Size of the folder hot_dog: ",     len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/hot_dog"), '*.jpg')))
print("Size of the folder pizza: ",     len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/pizza"), '*.jpg')))
print("Size of the folder sashimi: ",     len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/sashimi"), '*.jpg')))
print("Size of the folder sushi: ",     len(fnmatch.filter(os.listdir("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/sushi"), '*.jpg')))

In [ ]:
# CHECK AGAIN THE NUMBER OF IMAGES FOR EACH CLASSES
# COUNT THE NUMBER OF ELEMENTS IN EACH CLASS FOLDERS

# COUNT IMAGES AND CREATE A DICT
count_processed = 0
hist_dict_processed = {}

# ITERATE DIRECTORY
for path in os.listdir(DIR_TRAIN_TEST_DATA):
  if path == "train":

    print(path)
    for categories in os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path)):
      print("--", categories)

      for images in os.listdir(os.path.join(DIR_TRAIN_TEST_DATA, path, categories)):

        # CHECK IF CURRENT PATH IS A FILE
        if os.path.isfile(os.path.join(DIR_TRAIN_TEST_DATA, path, categories)):
          count_processed += 0
        else:
          # IF IS NOT A FILE, IS A DIRECTORY. GET THE FOLDER NAME
          number_of_elements_per_folder = len([entry for entry in os.listdir(DIR_TRAIN_TEST_DATA + "/" + path + "/" + categories)])
          # print(f"The folder called {path} has a number of element(s) equal to:", number_of_elements_per_folder)
          count_processed += 1
          hist_dict_processed[categories] = number_of_elements_per_folder
  else:
    break

print()
# print('Number of folders:', count_processed, "different classes of food.")

In [ ]:
# SHOW THE DATASET DISTRIBUTION WITH A HISTOGRAM PLOT ONLY FOR THE TRAINING SET
plt.figure(figsize=(15, 10))

# CONVERT THE DICT TO DATAFRAME
new_processed = pd.DataFrame([hist_dict_processed]).transpose().reset_index()
new_processed = new_processed.rename(columns={"index": "food_name", 0: "value_count"})

# SORT THE DATAFRAME BASED ON VALUE_COUNT
new_processed = new_processed.sort_values("value_count", ascending=False)
new_processed

# PLOT THE BAR AND THE TICKS
plt.bar(new_processed['food_name'], new_processed['value_count'], color='g')
plt.xticks(rotation=45)

# CREATING DATA ON WHICH BAR CHART WILL BE PLOT
x_food_name = list(new_processed['food_name'])
y_value_count = list(new_processed['value_count'])

# CALLING THE FUNCTION TO ADD VALUE LABELS
utils.addlabels(x_food_name, y_value_count)

### Now all the folders present a number of elements of 200 elements, solving the class unbalanced problem.

### Example of augmented images: in order to demonstrate how the data augmentation works, the following picture shows the application of brightness to a single image before applying it to multiple images in the dataset.

In [ ]:
# SHOW THE SIZE OF THE IMAGE: THE IMAGE HAS BEEN RESIZED
french_fries_shape = cv2.imread("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDatasetPreprocessed/french_fries/20160725_123734.jpg_RESIZED.jpg")
french_fries_shape.shape

In [ ]:
french_fries_brightness = cv2.imread("/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset/french_fries/cropfries46.jpg")
cv2_imshow(french_fries_brightness)

In [ ]:
# THE BRIGHTNESS FUNCTION RELIES ON A RANDOM CHOICE OF THE LOW AND HIGH VALUES.
# THEREFORE, EACH ITERATION CAN BE DIFFERENT FROM THE PREVIOUS ONE.
img_brightness = brightness(french_fries_brightness, 0.5, 1.3)
cv2_imshow(img_brightness)

In [ ]:
img_hue_image = hue_image(french_fries_brightness, 5)
cv2_imshow(img_hue_image)

In [ ]:
# SHOW THE RGB HISTOGRAM

# SET THE SIZE OF THE PICTURE
plt.figure(figsize=(10, 8)) 

# SETTING VALUES TO ROWS, COLUMN AND COLOR VARIABLE
# rows = 2
# columns = 2
colors = ('blue','green','red')
label = ("Blue", "Green", "Red")

# the container holding the two Axes have already been unpacked
# useful if just few Axes have been created
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8)) 

# FIRST PLOT ON THE LEFT
for count,color in enumerate(colors):
    histogram = cv2.calcHist(images = [french_fries_brightness], channels = [count], mask = None, histSize = [256], ranges = [0,256])
    # NORMALIZE IT
    histogram /= histogram.sum()
    ax1.plot(histogram,color = color, label=label[count]+str(" Pixels"))

ax1.set_title('Histogram Showing Number Of Pixels Belonging To Respective Pixel Intensity')
ax1.set_xlabel('Pixel Intensity - first plot')
ax1.set_ylabel('% Of Pixels - first plot')
ax1.legend(numpoints = 1, loc = "best")

for count,color in enumerate(colors):
    histogram = cv2.calcHist(images = [img_brightness], channels = [count], mask = None, histSize = [256], ranges = [0,256])
    # NORMALIZE IT
    histogram /= histogram.sum()
    ax2.plot(histogram,color = color, label=label[count]+str(" Pixels"))

ax2.set_title('Histogram Showing Number Of Pixels Belonging To Respective Pixel Intensity')
ax2.set_xlabel('Pixel Intensity - second plot')
ax2.set_ylabel('% Of Pixels - second plot')
ax2.legend(numpoints = 1, loc = "best")

plt.tight_layout()
plt.show()

A histogram represents the distribution of pixel intensities (whether color or grayscale) in an image. It can be visualized as a graph (or plot) that gives a high-level intuition of the intensity (pixel value) distribution. We are going to assume a RGB color space in this example, so these pixel values will be in the range of 0 to 255.
Each of the red, green and blue light levels is encoded as a number in the range 0.. 255, with 0 meaning zero light and 255 meaning maximum light. So for example (red=255, green=100, blue=0) is a color where red is maximum, green is medium, and blue is not present at all, resulting in a shade of orange.

Furthermore, the bins (0-255) are plotted on the x-axis. And the y-axis counts the number of pixels in each bin.

The majority of the pixels fall in the range of roughly 5 to 60: this implies that there are a lot of "black" and very few "white" pixels in the image.
In the right figure we can see that a bigger amount of darker pixel are created, so we can say that the brightness function created a darker image than the original one.

Check this out for data augmentation:
- https://towardsdatascience.com/complete-image-augmentation-in-opencv-31a6b02694f5

Prossimi passi:
- creare un for loop che vada a raccogliere tutte le immagini per ciascuna cartella SOLAMENTE PER IL TRAINING SET
- se la numerosità delle immagini è < 200, allora applica trasformazioni, salva nuove immagini trasformate e se si raggiungono le 200 immagini, procedi con la cartella successiva.
- se tutto ciò funziona, usa poi la data augmentation con ImageDataGenerator per creare un numero maggiore di immagini ma solo nella fase di modellazione! (check this out: https://stackoverflow.com/questions/51748514/does-imagedatagenerator-add-more-images-to-my-dataset; https://www.researchgate.net/post/How-many-images-does-Imagedatagenerator-generate-in-deep-learning#:~:text=Then%20the%20%22ImageDataGenerator%22%20will%20produce,there%20will%20be%20100%20iterations.)